In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10, 10]
np.random.seed(42)

In [2]:
from sklearn.datasets import fetch_20newsgroups_vectorized, fetch_20newsgroups

In [3]:
news_20 = fetch_20newsgroups()

In [9]:
news_20.data[:2]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [11]:
data = fetch_20newsgroups_vectorized()

In [16]:
noticias = data.data

In [17]:
noticias

<11314x130107 sparse matrix of type '<class 'numpy.float64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [18]:
noticias.shape

(11314, 130107)

In [19]:
data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [20]:
len(data.target_names)

20

Vemos que el dataset tiene 20 clusters naturales (20 paginas de noticias distintas)

In [21]:
from sklearn.cluster import MiniBatchKMeans

In [22]:
estimador = MiniBatchKMeans(n_clusters=20)
estimador.fit(noticias)
etiquetas_pred = estimador.labels_

In [23]:
etiquetas_pred

array([10, 13, 13, ...,  3,  8, 13])

# Medidas de evaluación externa

Son aquellas en las que las etiquetas de clusters son conocidas de antemano.

In [24]:
clases = data.target

In [25]:
from sklearn.metrics import homogeneity_completeness_v_measure, adjusted_rand_score

La función [homogeneity_completeness_v_measure](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html#sklearn.metrics.homogeneity_completeness_v_measure) nos devuelve una tupla con la homogeneidad, la completación y la medida v

In [27]:
homogeneity_completeness_v_measure(clases, etiquetas_pred)

(0.05879016390223451, 0.09237902927028353, 0.07185297689233971)

Vemos que estos clusters tienen mas completación que homogeneidad (es decir, los clusters son menos homogéneos que la homogeneidad de la distribución de las clases en clusters.

El v-measure tiene un rango de (0, 1) con lo cual estos clusters no son muy buenos.

Un problema que tiene el criterio de homogeneidad y completación es que no son invariantes del orden de las clases.

In [28]:
homogeneity_completeness_v_measure(etiquetas_pred, clases)

(0.09237902927028353, 0.05879016390223451, 0.07185297689233971)

Para datasets de menos de 1000 observaciones o para un número de clusters mayor que 10, se recomienda usar el Indice de Rand ajustado, [adjusted_rand_score](http://scikit-learn.org/stable/modules/clustering.html#adjusted-rand-index).

In [29]:
adjusted_rand_score(clases, etiquetas_pred)

0.026210490747621953

In [30]:
adjusted_rand_score(etiquetas_pred, clases)

0.026210490747621953

La métrica ARI tiene un rango de (-1,1) asi que pese a que este grupo de clusters no es el peor, no es muy bueno

Podemos usar medidas de evaluación externa para hacer validación cruzada como haríamos con cualquier problema de regresión/clasificación.

Métricas definidas en `cross_val_score` incluyen:

- `adjusted_rand_score`  
- `completeness_score` 	 
- `homogeneity_score` 	 
- `v_measure_score`

In [31]:
from sklearn.model_selection import cross_val_score

In [32]:
resultados = cross_val_score(X=noticias, y=clases, estimator=MiniBatchKMeans(), 
                             scoring="adjusted_rand_score", cv=5)

In [33]:
resultados.mean()

0.01602466535574484

# Medidas de evaluación interna
Dichas medidas se usan cuando las clases verdaderas no se conocen de antemano


In [36]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score

Estas medidas internas se usan en `sklearn` pasando dos argumentos, el dataset de entrenamiento y las etiquetas de los clusters.

En primer lugar tenemos el Coeficiente de Silueta [(silhouette_score)](http://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient)

In [37]:
silhouette_score(noticias, etiquetas_pred)

-0.08852047142766703

La métrica de calinski-Harabaz [(calinski-harabaz-score)](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabaz_score.html#sklearn.metrics.calinski_harabaz_score)es otra medida de evaluación interna. Nos da una medida de la dispersión dentro de un cluster y de la separación entre clusters. La diferencia con el coeficiente de silueta es que es más eficiente de calcular.

In [38]:
calinski_harabasz_score(noticias.todense(), etiquetas_pred)

56.1225600185843